In [3]:
import datasets
import matplotlib.pyplot as plt
import numpy as np
import nltk
import torch
import time
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [7]:
DATASET_DIR = './data'

esnli = datasets.load_dataset("esnli")

Found cached dataset esnli (/Users/lorenzorota/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc)


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
esnli["train"].shape[0]

549367

In [9]:
SUBSET_SIZE = 0.1

In [10]:
dataset = esnli

In [11]:
        train_size = int(dataset['train'].shape[0] * SUBSET_SIZE)
        val_size = int(dataset['validation'].shape[0] * SUBSET_SIZE)
        test_size = int(dataset['test'].shape[0] * SUBSET_SIZE)

In [14]:
dataset['validation'][0]

{'premise': 'Two women are embracing while holding to go packages.',
 'hypothesis': 'The sisters are hugging goodbye while holding to go packages after just eating lunch.',
 'label': 1,
 'explanation_1': 'The to go packages may not be from lunch.',
 'explanation_2': 'Just because two women are embracing, does not mean they are sisters.  Two women that are embracing are not necessarily hugging goodbye.',
 'explanation_3': 'Two women do not have to be sisters.  Embracing does not mean hugging goodbye.  The women do not have to have just finished lunch to be embracing.'}